ライブラリインポート

In [2]:
import pandas as pd
from sklearn import preprocessing
from tqdm import tqdm
import lightgbm as lgb
from slearn.metrics import accuracy_score, precision_score,recall_score, f1_score


ModuleNotFoundError: No module named 'slearn'

データセットの前処理

In [ ]:
#データセットのロード
dataset = pd.read_pickle('./data/dataset.pkl')
#使用する列名を指定
 #使用する列名を指定
resultCol = [
    '日付','raceId','枠番','馬番','horseId','性','年齢','斤量',
    'jokeyId','単勝','人気','trainerId','拠点','馬体重','体重増減',
    '出走間隔','ハンデ','着順','R','コース種','コース回り','距離','天気',
    '馬場','開催場所','グレード','制限'
]
recordCol = [
    'R','頭数','枠番','馬番','単勝','人気','着順','jockeyId','斤量',
    'タイム','着差','上り','馬体重','体重増減','出走間隔','コース種',
    'コース回り','距離','天気','馬場','開催場所','グレード','制限','ハンデ'
]
pedCol = ['pedId_' + str(i) for i in range(0,62)]
#前N走分戦績の列名を生成
recordCol9 = []
for i in range(1, 10):
    tmpList = list(map(lambda x: x + '_' + str(i), recordCol))
    recordCol9 += tmpList
#列名を合体
COLUMNS = resultCol + recordCol9 + pedCol
#データセット
dataset = dataset[COLUMNS]

In [ ]:
#ラベルエンコーディング関数の定義
def labelEncode(df, target, recflg=False):
    #複数列のラベルエンコーディング関数の定義
    def listEncoder(tdf, le, cols):
        #データフレームのコピー
        tdf_ = tdf.copy()
        #列名から値を取り出す
        encoList = []
        for col in cols:
            encoList += tdf_[col].unique().tolist()
        #エンコーダーを生成
        le.fit(encoList)
        #複数列分ループ
        for col in tqdm(cols, desc=cols[0]):
            #欠損データ以外の列を取り出す
            notNull = tdf_[col][tdf_[col].notnull()]
            #エンコード実行してindexをキーにデータフレームに書き込む
            tdf_[col] = pd.Series(le.transform(notNull), index=notNull.index)
            #エンコードした列はcategory列に変換
            tdf_[col] = tdf_[col].astype('category')
        return tdf_, le
    #データフレームのコピー
    tdf = df.copy()
    #ラベルエンコーダーをインスタンス
    le = preprocessing.LabelEncoder()
    #戦績かどうかで分岐
    if not recflg:
        #リストかどうかで分岐
        if type(target) == list:
            #エンコーダーの生成
            le.fit(tdf[target])
            #欠損データ以外の列を取り出す
            notNull = tdf[target][tdf[target].notnull()]
            #エンコード実行してindexをキーにデータフレームに書き込む
            tdf[target] = pd.Series(le.transform(notNull), index=notNull.index)
            #エンコードした列はcategory列に変換
            tdf[target] = tdf[target].astype('category')
        else:
            #戦績以外で複数データだったら複数列エンコードの実行
            tdf, le = listEncoder(tdf, le, target)
    else:
        #戦績データは列名にサフィックスを付与したリストを生成
        cols9 = [target] + [target + '_' + str(i) for i in range(1, 10)]
        #複数列エンコードの実行
        tdf, le = listEncoder(tdf, le, cols9)
    #データフレームとエンコーダーをreeturn
    return tdf, le

#データフレームコピー
df = dataset.copy()
#カテゴリ変数をラベルエンコード
horseList = ['horseId'] + ['pedId_' + str(i) for i in range(0,62)]
df, leHorse = labelEncode(df,horseList)
df, leGender = labelEncode(df,'性')
df, leTrainer = labelEncode(df,'trainerId')
df, leHomeBase = labelEncode(df,'拠点')
df, lejockey = labelEncode(df,'jockeyId',recflg=True)
df, leHandi = labelEncode(df,'ハンデ',recflg=True)
df, leType = labelEncode(df,'コース種',recflg=True)
df, leDir = labelEncode(df,'コース回り',recflg=True)
df, leWether = labelEncode(df,'天気',recflg=True)
df, leCondition = labelEncode(df,'馬場',recflg=True)
df, lePlace = labelEncode(df,'開催場所',recflg=True)
df, leGrade = labelEncode(df,'グレード', recflg=True)
df, leRegulation = labelEncode(df,'制限',recflg=True)
#量的変数の列名を生成
numericCols = ['年齢']
cols1 = ['枠番','馬番','単勝','人気','斤量','馬体重',
         '体重増減','出走間隔','着順','R','距離']
cols2 = ['頭数','着順','タイム','着差','上り']
numericCols += cols1
cols3 =cols1 + cols2
for i in range(1,10):
    numericCols += map(lambda x: x + '_' + str(i),cols3)
#量的変数に対して片変数を実行
for col in tqdm(numericCols):
    df[col] = df[col].astype(float)
    

学習実行

In [ ]:
#着順から正解列を生成
df['Accu'] = df['着順'].map(lambda x: 1 if x <= 3 else 0)
#日付をキーに訓練データと検証データに分割
sepdt = '2022/01/01'
train = df[df['日付']<sepdt]
test = df[df['日付']>=sepdt]

In [ ]:
#それぞれ教師データと訓練データに分割
train_x = train.drop(['日付','着順','raceId','Accu'],axis=1)
train_y = train['Accu']
test_x = train.drop(['日付','着順','raceId','Accu'],axis=1)
test_y = train['Accu']

In [ ]:
#モデルをインスタンスして学習の実行
model = lgb.LGBMClassifier()
model.fit(train_x,train_y)

学習モデルの評価
Accuracy score:単純正解率。
0を0と予測したものも正解とカウントさてるから単純に全部0と予想しても7割ぐらいの正解率になるからなし

Precision score:精度。1に分類したものが実際に1だった割合
Recall score:検出率。1のものを1として分類できた割合
F1 score:PrecisionとRecallを複合したスコア

In [ ]:
#検証データに対して予測実行
prad = model.predict(test_x)
#各種評価スコアの表示
print('Accuracy score\t: {}'.format(accuracy_score(prad,test_y)))
print('Precision score\t: {}'.format(precision_score(prad,test_y)))
print('Recall score\t: {}'.format(recall_score(prad,test_y)))
print('F1 score\t: {}'.format(f1_score(prad,test_y)))
